# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data Preparation

In [ ]:
foods = pd.read_csv('datasets/original data/food.csv')

In [ ]:
foods.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


In [ ]:
dataset = foods[['fdc_id', 'description']]
dataset

,fdc_id,description
0,319874,"HUMMUS, SABRA CLASSIC"
1,319875,"HUMMUS, SABRA CLASSIC"
2,319876,"HUMMUS, SABRA CLASSIC"
3,319877,Hummus
4,319878,Hummus
...,...,...
20536,1105893,"APPLES, FUJI"
20537,1105894,"APPLES, FUJI"
20538,1105895,"APPLES, FUJI"
20539,1105896,"APPLES, FUJI"


In [ ]:
dataset['description'] = dataset['description'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataset

,fdc_id,description
0,319874,"hummus, sabra classic"
1,319875,"hummus, sabra classic"
2,319876,"hummus, sabra classic"
3,319877,hummus
4,319878,hummus
...,...,...
20536,1105893,"apples, fuji"
20537,1105894,"apples, fuji"
20538,1105895,"apples, fuji"
20539,1105896,"apples, fuji"


In [ ]:
food_descriptions = dataset['description']

In [ ]:
food_descriptions

0               hummus, sabra classic
1               hummus, sabra classic
2               hummus, sabra classic
3                              hummus
4                              hummus
                     ...             
20536                    apples, fuji
20537                    apples, fuji
20538                    apples, fuji
20539                    apples, fuji
20540    apples, fuji, with skin, raw
Name: description, Length: 20541, dtype: object

In [ ]:
food_descriptions = food_descriptions.str.translate(str.maketrans('', '', string.punctuation))

In [ ]:
food_descriptions

0             hummus sabra classic
1             hummus sabra classic
2             hummus sabra classic
3                           hummus
4                           hummus
                   ...            
20536                  apples fuji
20537                  apples fuji
20538                  apples fuji
20539                  apples fuji
20540    apples fuji with skin raw
Name: description, Length: 20541, dtype: object

In [ ]:
type(food_descriptions)

pandas.core.series.Series

In [ ]:
sentences = food_descriptions.to_numpy()
sentences

array(['hummus sabra classic', 'hummus sabra classic',
       'hummus sabra classic', ..., 'apples fuji', 'apples fuji',
       'apples fuji with skin raw'], dtype=object)

In [ ]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
tokenized_sent

[['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus'],
 ['hummus'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus'],
 ['hummus'],
 ['hummus', 'other'],
 ['hummus', 'other'],
 ['hummus', 'other'],
 ['hummus', 'other'],
 ['hummus', 'other'],
 ['hummus'],
 ['hummus'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'sabra', 'classic'],
 ['hummus', 'nfy12140o'],
 ['hummus', 'nfy12140p'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus'],
 ['hummus', 'nfy12140q'],
 ['hummus'

# Infersent model with GloVe embeddings

In [ ]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  16.7M      0  0:00:08  0:00:08 --:--:-- 19.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5155k      0  0:06:52  0:06:52 --:--:-- 5235k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 145.0MB 88kB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
!pip install models

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
import torch

In [ ]:
from models import InferSent

In [ ]:
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [ ]:
model.build_vocab(sentences, tokenize=True)

Found 1518(/10810) words with w2v vectors
Vocab size : 1518


In [ ]:
query = 'hummus sabra classic'
query_vec = model.encode(query)[0]
len(query_vec)

/content/models.py:196: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


4096

In [ ]:
embeddings = model.encode(sentences, tokenize=True)

/content/models.py:196: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


In [ ]:
embeddings

array([[-0.02874381,  0.07464062, -0.1518522 , ..., -0.00808924,
         0.02951451,  0.03568656],
       [-0.02874381,  0.07464062, -0.1518522 , ..., -0.00808924,
         0.02951451,  0.03568656],
       [-0.02874381,  0.07464062, -0.1518522 , ..., -0.00808924,
         0.02951451,  0.03568656],
       ...,
       [-0.0092328 ,  0.04943122, -0.07869949, ..., -0.01258786,
         0.01049058,  0.04160625],
       [-0.0092328 ,  0.04943122, -0.07869949, ..., -0.01258786,
         0.01049058,  0.04160625],
       [-0.0092328 ,  0.04943122,  0.11749992, ...,  0.03581332,
         0.01712248,  0.05927012]], dtype=float32)

In [ ]:
len(embeddings)

20541

In [ ]:
len(food_descriptions)

20541

In [ ]:
df_embeddings = pd.DataFrame(embeddings)

In [ ]:
df_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,-0.005039,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
1,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,-0.005039,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
2,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,-0.005039,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
3,-0.028744,0.074641,-0.173372,-0.037082,0.009704,0.026854,0.054186,-0.055775,0.005188,0.013112,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.025629,-0.038263,0.102770,-0.009750,-0.012693,0.021990,-0.043041,-0.087955,0.119307,0.037840,0.084635,-0.055724,0.057714,0.119243,-0.100898,0.056552,-0.015683,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,-0.013334,-0.013846,...,0.031487,-0.104898,-0.027031,0.111876,0.021328,0.040361,0.003525,-0.00409,0.049973,0.181910,-0.114019,0.017652,-0.015042,-0.064917,0.038925,-0.047457,-0.054332,-0.010818,-0.01958,0.008602,-0.010044,-0.091156,-0.071367,0.044314,-0.010373,-0.030837,-0.062794,0.060004,0.081073,-0.028217,-0.055255,0.000593,-0.072606,-0.078184,-0.003411,0.006504,-0.090868,-0.012588,-0.067958,0.018408
4,-0.028744,0.074641,-0.173372,-0.037082,0.009704,0.026854,0.054186,-0.055775,0.005188,0.013112,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.025629,-0.038263,0.102770,-0.009750,-0.012693,0.021990,-0.043041,-0.087955,0.119307,0.037840,0.084635,-0.055724,0.057714,0.119243,-0.100898,0.056552,-0.015683,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,-0.013334,-0.013846,...,0.031487,-0.104898,-0.027031,0.111876,0.021328,0.040361,0.003525,-0.00409,0.049973,0.181910,-0.114019,0.017652,-0.015042,-0.064917,0.038925,-0.047457,-0.054332,-0.010818,-0.01958,0.008602,-0.010044,-0.091156,-0.071367,0.044314,-0.010373,-0.030837,-0.062794,0.060004

In [ ]:
df_embeddings.insert(loc=0, column='fdc_id', value=ids_df)
df_embeddings

,fdc_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,319874,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
1,319875,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
2,319876,-0.028744,0.074641,-0.151852,-0.015479,0.063549,0.051299,0.054186,-0.007078,0.133623,0.026318,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.013458,-0.038263,0.060156,-0.009750,-0.024308,0.062596,-0.043041,-0.087955,0.095171,0.037840,0.059895,-0.024885,0.057714,0.119243,-0.006012,0.056552,0.017903,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,0.106856,...,0.021044,-0.005886,-0.027031,0.168544,0.021328,0.040361,0.049740,-0.00409,0.091483,0.204793,-0.080080,0.017652,-0.013225,0.048373,0.074572,0.019896,0.067744,-0.010818,-0.00704,-0.010579,0.006823,-0.091156,-0.027813,0.044314,0.030324,0.011751,0.027296,0.019571,0.081073,-0.028217,-0.025439,0.000593,0.025796,-0.099384,0.004749,0.014027,-0.034328,-0.008089,0.029515,0.035687
3,319877,-0.028744,0.074641,-0.173372,-0.037082,0.009704,0.026854,0.054186,-0.055775,0.005188,0.013112,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.025629,-0.038263,0.102770,-0.009750,-0.012693,0.021990,-0.043041,-0.087955,0.119307,0.037840,0.084635,-0.055724,0.057714,0.119243,-0.100898,0.056552,-0.015683,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,-0.013334,...,0.031487,-0.104898,-0.027031,0.111876,0.021328,0.040361,0.003525,-0.00409,0.049973,0.181910,-0.114019,0.017652,-0.015042,-0.064917,0.038925,-0.047457,-0.054332,-0.010818,-0.01958,0.008602,-0.010044,-0.091156,-0.071367,0.044314,-0.010373,-0.030837,-0.062794,0.060004,0.081073,-0.028217,-0.055255,0.000593,-0.072606,-0.078184,-0.003411,0.006504,-0.090868,-0.012588,-0.067958,0.018408
4,319878,-0.028744,0.074641,-0.173372,-0.037082,0.009704,0.026854,0.054186,-0.055775,0.005188,0.013112,-0.014443,0.028209,-0.051063,0.000696,-0.002273,0.025629,-0.038263,0.102770,-0.009750,-0.012693,0.021990,-0.043041,-0.087955,0.119307,0.037840,0.084635,-0.055724,0.057714,0.119243,-0.100898,0.056552,-0.015683,-0.014131,-0.020641,0.098027,0.035690,0.046750,0.097157,-0.013334,...,0.031487,-0.104898,-0.027031,0.111876,0.021328,0.040361,0.003525,-0.00409,0.049973,0.181910,-0.114019,0.017652,-0.015042,-0.064917,0.038925,-0.047457,-0.054332,-0.010818,-0.01958,0.008602,-0.010044,-0.091156,-0.071367,0.044314,-0.010373,-0.030837,-0.062794,0.060004,0.081073,-

In [ ]:
df_embeddings.to_csv('datasets/food_embeddings.csv')

# New datasets with added word embeddings

## PCA - 20 columns

In [ ]:
embeddings = pd.read_csv('datasets/food_embeddings.csv')

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5) 
pca_20 = pca.fit(embeddings).transform(embeddings)
pca_20 

array([[-1.39558995e+05,  7.63843953e+03,  4.45465795e-01,
         7.02724618e-01,  6.53884221e-01],
       [-1.39557977e+05,  7.63745857e+03,  4.45570091e-01,
         7.02719812e-01,  6.53893663e-01],
       [-1.39556958e+05,  7.63647761e+03,  4.45674388e-01,
         7.02715006e-01,  6.53903106e-01],
       ...,
       [ 6.46709579e+05,  1.93316724e+03, -4.58607753e-01,
         2.48127174e-01,  9.61693809e-01],
       [ 6.46710598e+05,  1.93218628e+03, -4.58503457e-01,
         2.48122368e-01,  9.61703251e-01],
       [ 6.46711617e+05,  1.93120514e+03, -1.90709865e+00,
        -4.22631156e-01,  2.66340382e-01]])

In [ ]:
ids = dataset['fdc_id']
ids

0         319874
1         319875
2         319876
3         319877
4         319878
          ...   
20536    1105893
20537    1105894
20538    1105895
20539    1105896
20540    1105897
Name: fdc_id, Length: 20541, dtype: int64

In [ ]:
ids_df = pd.DataFrame(ids)
emb_df = pd.DataFrame(pca_20)
emb_df.insert(loc=0, column='fdc_id', value=ids_df)
emb_df

,fdc_id,0,1,2,3,4
0,319874,-139558.995341,7638.439525,0.445466,0.702725,0.653884
1,319875,-139557.976653,7637.458569,0.445570,0.702720,0.653894
2,319876,-139556.957965,7636.477613,0.445674,0.702715,0.653903
3,319877,-139555.939272,7635.496843,2.249480,0.656333,0.574526
4,319878,-139554.920584,7634.515887,2.249584,0.656329,0.574536
...,...,...,...,...,...,...
20536,1105893,646707.542106,1935.129148,-0.458816,0.248137,0.961675
20537,1105894,646708.560794,1934.148192,-0.458712,0.248132,0.961684
20538,1105895,646709.579482,1933.167236,-0.458608,0.248127,0.961694
20539,1105896,646710.598170,1932.186280,-0.458503,0.248122,0.961703


In [ ]:
least_20 = pd.read_csv('datasets/average datasets/byColumns/20_columns.csv')
least_20

,Unnamed: 0,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264
0,319877,19.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.3,1.98,1.28,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,319878,0.00,46.0,75.9,326.0,1.50,188.0,2.33,0.380,1.210,0.0,0.00,0.00,446.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,319882,18.70,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.1,1.99,1.29,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,319883,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,319884,0.00,57.0,81.4,327.0,1.47,190.0,2.96,0.394,1.200,0.0,0.00,0.00,444.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,1105893,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
13101,1105894,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
13102,1105895,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
13103,1105896,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,83.8,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [ ]:
least_20 = least_20.rename(columns={'Unnamed: 0':'fdc_id'})

In [ ]:
cols_median

,fdc_id,1004,1003,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1093,1002,1167,1175,1165,1166,1103,1079
0,319877,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,319878,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,319882,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,319884,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,319892,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13098,1105891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
13099,1105892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
13100,1105893,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13102,1105895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
least_20_with_embeddings = pd.merge(least_20, emb_df, on='fdc_id')
least_20_with_embeddings

,fdc_id,1004,1003,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1093,1002,1167,1175,1165,1166,1103,1079,0,1,2,3,4
0,319877,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,-139555.939272,7635.496843,2.249480,0.656333,0.574526
1,319878,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139554.920584,7634.515887,2.249584,0.656329,0.574536
2,319882,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,-139550.845832,7630.592063,2.250001,0.656309,0.574573
3,319884,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139548.808456,7628.630151,2.250210,0.656300,0.574592
4,319892,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,-139540.658952,7620.782503,2.251044,0.656261,0.574668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,1105891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,646705.504729,1937.091060,-0.459025,0.248146,0.961656
7612,1105892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,646706.523417,1936.110104,-0.458921,0.248142,0.961665
7613,1105893,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,646707.542106,1935.129148,-0.458816,0.248137,0.961675
7614,1105895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,646709.579482,1933.167236,-0.458608,0.248127,0.961694


In [ ]:
least_20_with_embeddings.to_csv('datasets/average datasets/byColumnsWithEmbeddings/20_columns_embeddings.csv')

## PCA - 40 columns

In [ ]:
pca = PCA(n_components=10) 
pca_40 = pca.fit(embeddings).transform(embeddings)
pca_40 

array([[-1.39558995e+05,  7.63843953e+03,  4.45465798e-01, ...,
         9.68367918e-02, -3.02407614e-01,  4.84334490e-01],
       [-1.39557977e+05,  7.63745857e+03,  4.45570094e-01, ...,
         9.68407919e-02, -3.02413783e-01,  4.84309601e-01],
       [-1.39556958e+05,  7.63647761e+03,  4.45674391e-01, ...,
         9.68447921e-02, -3.02419952e-01,  4.84284712e-01],
       ...,
       [ 6.46709579e+05,  1.93316724e+03, -4.58607755e-01, ...,
         2.59487511e-02,  2.78505767e-02,  1.26558045e-01],
       [ 6.46710598e+05,  1.93218628e+03, -4.58503458e-01, ...,
         2.59527512e-02,  2.78444078e-02,  1.26533156e-01],
       [ 6.46711617e+05,  1.93120514e+03, -1.90709866e+00, ...,
         7.57831605e-02, -6.06214308e-02,  3.45379603e-02]])

In [ ]:
emb_df = pd.DataFrame(pca_40)
emb_df.insert(loc=0, column='fdc_id', value=ids_df)
emb_df

,fdc_id,0,1,2,3,4,5,6,7,8,9
0,319874,-139558.995341,7638.439525,0.445466,0.702725,0.653914,0.503104,-0.360152,0.096837,-0.302408,0.484334
1,319875,-139557.976653,7637.458569,0.445570,0.702720,0.653924,0.503125,-0.360165,0.096841,-0.302414,0.484310
2,319876,-139556.957965,7636.477613,0.445674,0.702715,0.653933,0.503146,-0.360178,0.096845,-0.302420,0.484285
3,319877,-139555.939272,7635.496843,2.249480,0.656334,0.574553,0.448895,-0.018260,0.243760,0.003203,0.468584
4,319878,-139554.920584,7634.515887,2.249584,0.656329,0.574562,0.448916,-0.018272,0.243764,0.003197,0.468559
...,...,...,...,...,...,...,...,...,...,...,...
20536,1105893,646707.542106,1935.129148,-0.458816,0.248137,0.961666,0.456682,-0.118083,0.025941,0.027863,0.126608
20537,1105894,646708.560794,1934.148192,-0.458712,0.248132,0.961675,0.456703,-0.118096,0.025945,0.027857,0.126583
20538,1105895,646709.579482,1933.167236,-0.458608,0.248127,0.961685,0.456724,-0.118109,0.025949,0.027851,0.126558
20539,1105896,646710.598170,1932.186280,-0.458503,0.248122,0.961694,0.456745,-0.118122,0.025953,0.027844,0.126533


In [ ]:
least_40 = pd.read_csv('datasets/average datasets/byColumns/40_columns.csv')
least_40 = least_40.rename(columns={'Unnamed: 0':'fdc_id'})
least_40_with_embeddings = pd.merge(least_40, emb_df, on='fdc_id')

In [ ]:
least_40_with_embeddings

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,1165,0,1,2,3,4,5,6,7,8,9
0,319877,19.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.3,1.98,1.28,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,-139555.939272,7635.496843,2.249480,0.656334,0.574553,0.448895,-0.018260,0.243760,0.003203,0.468584
1,319878,0.00,46.0,75.9,326.0,1.50,188.0,2.33,0.380,1.210,0.0,0.00,0.00,446.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,-139554.920584,7634.515887,2.249584,0.656329,0.574562,0.448916,-0.018272,0.243764,0.003197,0.468559
2,319882,18.70,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.1,1.99,1.29,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,-139550.845832,7630.592063,2.250001,0.656310,0.574600,0.449000,-0.018324,0.243780,0.003172,0.468459
3,319883,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,-139549.827144,7629.611107,2.250106,0.656305,0.574609,0.449020,-0.018337,0.243784,0.003166,0.468434
4,319884,0.00,57.0,81.4,327.0,1.47,190.0,2.96,0.394,1.200,0.0,0.00,0.00,444.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,-139548.808456,7628.630151,2.250210,0.656300,0.574619,0.449041,-0.018350,0.243788,0.003160,0.468409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,1105893,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,646707.542106,1935.129148,-0.458816,0.248137,0.961666,0.456682,-0.118083,0.025941,0.027863,0.126608
13101,1105894,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,646708.560794,1934.148192,-0.458712,0.248132,0.961675,0.456703,-0.118096,0.025945,0.027857,0.126583
13102,1105895,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,646709.579482,1933.167236,-0.458608,0.248127,0.961685,0.456724,-0.118109,0.025949,0.027851,0.126558
13103,1105896,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,83.8,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,646710.598170,1932.186280,-0.458503,0.248122,0.961694,0.456745,-0.118122,0.025953,0.027844,0.126533


In [ ]:
least_40_with_embeddings.to_csv('datasets/average datasets/byColumnsWithEmbeddings/40_columns_embeddings.csv')

## PCA - 80 columns

In [ ]:
pca = PCA(n_components=20) 
pca_df = pca.fit(embeddings).transform(embeddings)
emb_df = pd.DataFrame(pca_df)
emb_df.insert(loc=0, column='fdc_id', value=ids_df)
least_80 = pd.read_csv('datasets/average datasets/byColumns/80_columns.csv')
least_80 = least_80.rename(columns={'Unnamed: 0':'fdc_id'})
least_80_with_embeddings = pd.merge(least_80, emb_df, on='fdc_id')

In [ ]:
least_80_with_embeddings

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1277,1292,1258,1293,1405,1010,1170,1109,1177,1178,1312,2009,1334,1257,1100,1107,1223,1224,1210,1211,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,319877,19.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.3,1.98,1.28,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139555.939272,7635.496843,2.249480,0.656333,0.574551,0.448896,-0.018277,0.243780,0.002969,0.468625,-0.359804,0.063687,0.352992,0.168597,-0.084233,-0.021254,0.060233,-0.468665,-0.616268,-0.146100
1,319878,0.00,46.0,75.9,326.0,1.50,188.0,2.33,0.380,1.210,0.0,0.00,0.00,446.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139554.920584,7634.515887,2.249584,0.656329,0.574561,0.448917,-0.018290,0.243784,0.002962,0.468600,-0.359791,0.063674,0.352992,0.168603,-0.084209,-0.021250,0.060244,-0.468658,-0.616257,-0.146110
2,319882,18.70,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.1,1.99,1.29,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139550.845832,7630.592063,2.250001,0.656309,0.574599,0.449000,-0.018341,0.243800,0.002938,0.468501,-0.359740,0.063622,0.352994,0.168629,-0.084111,-0.021234,0.060287,-0.468631,-0.616213,-0.146152
3,319883,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139549.827144,7629.611107,2.250106,0.656305,0.574608,0.449021,-0.018354,0.243804,0.002932,0.468476,-0.359728,0.063609,0.352994,0.168636,-0.084086,-0.021230,0.060298,-0.468624,-0.616202,-0.146163
4,319884,0.00,57.0,81.4,327.0,1.47,190.0,2.96,0.394,1.200,0.0,0.00,0.00,444.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139548.808456,7628.630151,2.250210,0.656300,0.574618,0.449042,-0.018367,0.243808,0.002925,0.468451,-0.359715,0.063596,0.352994,0.168642,-0.084061,-0.021226,0.060309,-0.468617,-0.616191,-0.146173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,1105893,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,646707.542106,1935.129148,-0.458816,0.248137,0.961668,0.456681,-0.118076,0.025899,0.027928,0.126579,-0.212163,0.506348,-0.288480,0.113097,-0.087882,-0.182702,-0.198705,0.227990,0.052383,-0.131744
13101,1105894,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,646708.560794,1934.148192,-0.458712,0.248132,0.961677,0.456701,-0.118089,0.025903,0.027922,0.126555,-0.212150,0.506335,-0.288480,0.113103,-0.087858,-0.182698,-0.198694,0.227997,0.052394,-0.131754
13102,1105895,0.00,0.0,0.0,0.0,0.00,0.0,0.00

In [ ]:
least_80_with_embeddings.to_csv('datasets/average datasets/byColumnsWithEmbeddings/80_columns_embeddings.csv')

## PCA - 120 columns

In [ ]:
pca = PCA(n_components=30) 
pca_df = pca.fit(embeddings).transform(embeddings)
emb_df = pd.DataFrame(pca_df)
emb_df.insert(loc=0, column='fdc_id', value=ids_df)
least_120 = pd.read_csv('datasets/average datasets/byColumns/120_columns.csv')
least_120 = least_120.rename(columns={'Unnamed: 0':'fdc_id'})
least_120_with_embeddings = pd.merge(least_120, emb_df, on='fdc_id')

In [ ]:
least_120_with_embeddings

,fdc_id,1004,1087,1090,1092,1095,1091,1089,1098,1101,1051,1007,1002,1093,1079,1003,1009,1102,1265,1266,1264,1315,1316,1314,1094,1097,1137,1146,1300,1299,1404,1267,1273,1323,1167,1271,1304,1166,1175,1263,...,1180,1194,1128,1195,1005,1075,1196,1197,1198,1084,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,319877,19.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.3,1.98,1.28,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139555.939272,7635.496843,2.249480,0.656333,0.574551,0.448896,-0.018277,0.243781,0.002971,0.468627,-0.359818,0.063704,0.353036,0.168627,-0.084416,-0.019647,0.061010,-0.468975,-0.613797,-0.142394,0.380040,0.137038,0.343424,-0.024100,0.212986,-0.399058,-0.357483,0.271266,0.549202,-0.132740
1,319878,0.00,46.0,75.9,326.0,1.50,188.0,2.33,0.380,1.210,0.0,0.00,0.00,446.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139554.920584,7634.515887,2.249584,0.656329,0.574561,0.448917,-0.018290,0.243785,0.002965,0.468602,-0.359805,0.063691,0.353036,0.168633,-0.084392,-0.019643,0.061021,-0.468968,-0.613786,-0.142405,0.380045,0.137037,0.343410,-0.024088,0.212994,-0.399053,-0.357474,0.271261,0.549210,-0.132739
2,319882,18.70,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,56.1,1.99,1.29,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139550.845832,7630.592063,2.250001,0.656309,0.574599,0.449000,-0.018341,0.243801,0.002940,0.468503,-0.359754,0.063639,0.353037,0.168659,-0.084293,-0.019627,0.061064,-0.468941,-0.613742,-0.142449,0.380066,0.137037,0.343357,-0.024039,0.213026,-0.399035,-0.357438,0.271242,0.549241,-0.132735
3,319883,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139549.827144,7629.611107,2.250106,0.656305,0.574608,0.449021,-0.018354,0.243805,0.002934,0.468478,-0.359741,0.063626,0.353038,0.168666,-0.084269,-0.019623,0.061075,-0.468934,-0.613731,-0.142460,0.380071,0.137037,0.343343,-0.024027,0.213034,-0.399031,-0.357430,0.271237,0.549249,-0.132734
4,319884,0.00,57.0,81.4,327.0,1.47,190.0,2.96,0.394,1.200,0.0,0.00,0.00,444.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-139548.808456,7628.630151,2.250210,0.656300,0.574618,0.449042,-0.018367,0.243809,0.002928,0.468453,-0.359729,0.063613,0.353038,0.168672,-0.084244,-0.019619,0.061085,-0.468927,-0.613720,-0.142471,0.380077,0.137036,0.343330,-0.024015,0.213041,-0.399026,-0.357421,0.271232,0.549257,-0.132733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,1105893,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,646707.542106,1935.129148,-0.458816,0.248137,0.961668,0.456681,-0.118076,0.025900,0.027929,0.126578,-0.212165,0.506342,-0.288497,0.113084,-0.087944,-0.182973,-0.198585,0.228102,0.051490,-0.132350,-0.359027,0.006197,-0.489643,-0.608052,-0.074036,0.286011,-0.146373,0.310294,0.291382,0.085531
13101,1105894,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
least_120_with_embeddings.to_csv('datasets/average datasets/byColumnsWithEmbeddings/120_columns_embeddings.csv')

## PCA - 198 columns

In [ ]:
pca = PCA(n_components=50) 
pca_df = pca.fit(df_embeddings).transform(df_embeddings)
emb_df = pd.DataFrame(pca_df)
emb_df.insert(loc=0, column='fdc_id', value=ids_df)
least_198 = pd.read_csv('datasets/average datasets/byColumns/average_dataset_198.csv')
least_198 = least_198.rename(columns={'Unnamed: 0':'fdc_id'})
least_198_with_embeddings = pd.merge(least_198, emb_df, on='fdc_id')
least_198_with_embeddings

,fdc_id,1002,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1024,1032,1039,1050,1051,1062,1063,1075,1079,1082,1084,1085,1087,1089,1090,1091,1092,1093,1094,1095,1097,1098,1100,1101,1102,1103,1105,...,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,319877,1.28,0.00,19.00,0.0,1.98,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,-0.359817,0.063704,0.353035,0.168626,-0.084408,-0.019656,0.060990,-0.468931,-0.613675,-0.142368,0.379975,0.137206,0.343752,-0.024434,0.212981,-0.402506,-0.358474,0.283379,0.534259,-0.146894,-0.006226,0.203522,0.073713,-0.022458,-0.061995,0.053995,-0.104444,0.178987,-0.412227,0.097564,-0.081371,-0.165857,-0.356845,-0.004772,0.031287,0.220120,0.019949,0.090595,0.155663,-0.131060
1,319878,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,2.33,75.9,188.0,326.0,446.0,0.0,1.50,0.0,0.380,0.0,1.210,0.0,0.0,0.0,...,-0.359805,0.063691,0.353035,0.168632,-0.084384,-0.019652,0.061000,-0.468925,-0.613664,-0.142379,0.379980,0.137206,0.343739,-0.024421,0.212988,-0.402501,-0.358465,0.283375,0.534267,-0.146892,-0.006216,0.203522,0.073721,-0.022457,-0.061990,0.054001,-0.104438,0.178986,-0.412225,0.097567,-0.081375,-0.165854,-0.356843,-0.004771,0.031286,0.220123,0.019949,0.090599,0.155668,-0.131056
2,319882,1.29,0.00,18.70,0.0,1.99,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,-0.359754,0.063639,0.353036,0.168658,-0.084286,-0.019636,0.061044,-0.468898,-0.613620,-0.142423,0.380001,0.137205,0.343685,-0.024372,0.213020,-0.402483,-0.358430,0.283357,0.534299,-0.146888,-0.006176,0.203524,0.073753,-0.022453,-0.061971,0.054023,-0.104415,0.178980,-0.412219,0.097577,-0.081390,-0.165841,-0.356835,-0.004766,0.031282,0.220138,0.019946,0.090611,0.155688,-0.131040
3,319883,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,-0.359741,0.063626,0.353037,0.168664,-0.084261,-0.019632,0.061054,-0.468891,-0.613609,-0.142434,0.380007,0.137205,0.343672,-0.024360,0.213028,-0.402478,-0.358421,0.283352,0.534307,-0.146887,-0.006166,0.203524,0.073761,-0.022452,-0.061966,0.054029,-0.104410,0.178979,-0.412217,0.097579,-0.081394,-0.165838,-0.356833,-0.004764,0.031281,0.220142,0.019945,0.090614,0.155693,-0.131035
4,319884,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,2.96,81.4,190.0,327.0,444.0,0.0,1.47,0.0,0.394,0.0,1.200,0.0,0.0,0.0,...,-0.359728,0.063613,0.353037,0.168671,-0.084236,-0.019628,0.061065,-0.468884,-0.613598,-0.142445,0.380012,0.137205,0.343659,-0.024348,0.213036,-0.402474,-0.358412,0.283348,0.534315,-0.146886,-0.006156,0.203525,0.073769,-0.022451,-0.061961,0.054034,-0.104404,0.178978,-0.412215,0.097582,-0.081398,-0.165835,-0.356831,-0.004763,0.031280,0.220146,0.019945,0.090617,0.155698,-0.131031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,1105893,0.00,0.00,0.20,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,-0.212166,0.506342,-0.288495,0.113080,-0.087950,-0.182960,-0.198626,0.228190,0.051409,-0.132333,-0.359060,0.006172,-0.489187,-0.608642,-0.073537,0.283162,-0.146314,0.322133,0.285970,0.096542,0.139419,0.066745,0.009758,-0.016907,-0.097714,0.033979,-0.270718,-0.144216,0.118889,-0.373315,0.124977,0.215571,0.014635,0.200687,0.039840,-0.

In [ ]:
least_198_with_embeddings.to_csv('datasets/average datasets/byColumnsWithEmbeddings/198_columns_embeddings.csv')